In [56]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)

from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import tensor_basis,spinful_fermion_basis_1d,boson_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library
#
##### setting up parameters for simulation
# physical parameters
l = 1
L_f = 3 # system size f
L_b = L_f-1 # system size b
Nf, Nb = 2, l # number of fermions, bosons
N = Nf + Nb # total number of particles
# define time-dependent perturbation

###### create the basis
# build the two bases to tensor together to a bose-fermi mixture
basis_b1=boson_basis_1d(L_b,Nb=[i for i in range(2*l+1)]) # boson basis
basis_b2=boson_basis_1d(L_b,Nb=[i for i in range(2*l+1)]) # boson basis
basis_f=spinful_fermion_basis_1d(L_f,Nf = [(2,2),(3,1),(1,3)],double_occupancy=True)

basis=tensor_basis(basis_b1,basis_b2,basis_f) # BFM
#
#print(basis_b1)
#print(basis_f)
##### create model

############################################## target  hamiltonian
a = 1.0
e = 1.0
m = 1.0
lam = 0.5

bosonic_part_LGT = [[a*e**2/4.,i,i,i,i] for i in range(L_b)] # b^dagger*b^dagger*b*b
mass_f = [[m+1.0/a,i,i] for i in range(L_f)] # f mass term
interaction_part_LGT = [[1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction
gauge_violation = [[lam,i,(i+1)] for i in range(L_f-1)] #gauge violating term


b_0 = [[-0.5*e,0]]
d_0 = [[+0.5*e,0]]
b_1 = [[+0.5*e,1]]
d_1 = [[-0.5*e,1]]
rho_1 = [[e,1]]


static_gauss_law = [
    
    ["n|||", b_0],
    ["|n||", d_0],
    ["n|||", b_1],
    ["|n||", d_1],
    ["||n|", rho_1],
    ["|||n", rho_1]

]    


    
    

#
static_target = [
            ["++--|||", bosonic_part_LGT], # b^dagger*b^dagger*b*b
            ["|++--||", bosonic_part_LGT], # b^dagger*b^dagger*b*b

            ["||+|-", mass_f], # f mass term
            ["||-|+", mass_f], # f mass term


            ["+|-|+|-", interaction_part_LGT], #boson-fermion interaction
            ["-|+|-|+", interaction_part_LGT], #boson-fermion interaction


]
static_violation = [
["||+|-", gauge_violation],
["||-|+", gauge_violation]

]

################################################
###### set up Hamiltonian and initial states
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

H_violation = hamiltonian(static_violation,dynamic_list = [],basis=basis,**no_checks)
H_target = hamiltonian(static_target,dynamic_list = [],basis=basis,**no_checks)
Gauss_law = hamiltonian(static_gauss_law, dynamic_list = [], basis=basis, **no_checks)

# define initial Fock state through strings
s_f1 = "101"
s_f2 = "101"
s_b1 = "11"
s_b2 = "11"

# basis.index accepts strings and returns the index which corresponds to that state in the basis list
i_0 = basis.index(s_b1,s_b2,s_f1,s_f2) # find index of product state in basis
psi_0 = np.zeros(basis.Ns) # allocate space for state
psi_0[i_0] = 1.0 # set MB state to be the given product state
#print("H-space size: {:d}, initial state: |{:s}>|{:s}>".format(basis.Ns,s_b,s_f))
#
###### time evolve initial state and measure energy
times = np.linspace(0.0,10,100)
def extract_energy(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_0,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value(psi_t3) + Gauss_law.expt_value(psi_t3)**2    
    return np.real(cost_function[-1])

In [57]:
from scipy.optimize import brute
from scipy.optimize import minimize

brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy, brute_ranges, finish = minimize)

In [58]:
print(resbrute)

[0. 0.]
